First enstablish the SSL via PuTTY, connecting to the tool-login.wmflabs.org, from port 4711 to the destination enwiki.labsdb (at port 3306). Insert your credentials below:

In [1]:
import pymysql
import datetime
import pandas as pd

wikiconn  = pymysql.connect(host='localhost',
                            user='u15068',                 # get from replica.my.cnf
                            password='G5V2xrWbdFTSkyoU',   # get from replica.my.cnf
                            port=4711,
                            db='enwiki_p',
                            charset='utf8')
wikiCur = wikiconn.cursor()

In [2]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [47]:
def PrintableTimestamp():
    now = datetime.datetime.now().time()
    return "[{:02d}:{:02d}:{:02d}]".format(now.hour,now.minute,now.second)

def BatchQueryFromList(listOfIDs,columnThatContainsIDs,tableToQuery,listOfColumnsToRetrieve):
    '''listOfIDs is a list of keys to retrieve from an SQL table
    columnThatContainsIDs is the column in that table that stores the keys
    tableToQuery is the table containing the information you want
    listOfColumnsToRetrieve is a list containing the SQL DB table column names you want to retrieve
    Returns a pandas dataframe with column names equivalent to those in the DB you queried'''
    baseQ = "SELECT {} FROM {} WHERE {} IN ({})"
    colsForQ = ",".join(listOfColumnsToRetrieve)
    idChunks = chunks(listOfIDs, 1000)
    result_df = pd.DataFrame()
    for ix, chunk in enumerate(idChunks):
        print(PrintableTimestamp() + " fetching chunk {}".format(ix))
        idsInChunk = ",".join(map(str,chunk))
        preppedQ = baseQ.format(colsForQ,tableToQuery,columnThatContainsIDs,idsInChunk)
        wikiCur.execute(preppedQ)
        if not result_df.empty:
            result_df.append(list(wikiCur.fetchall()))
            print("here I'm appending", result_df[(ix*1000-3):])
        else:
            result_df = pd.DataFrame(list(wikiCur.fetchall()),columns=listOfColumnsToRetrieve)
            print("that was the first touch")
    return result_df

In [48]:
# get IDS for EPAs from github file. For NEPAs: ../Sampling/071816_batch1_sampled_ids.csv
ids2Q = pd.read_csv("../WikiSQL/070916_edit_protected_articles.csv")  # For EPAs: ../WikiSQL/070916_edit_protected_articles.csv



In [49]:
ids2Q[:3]

,page_id,page_namespace,page_title,page_restrictions,page_counter,page_is_redirect,page_is_new,page_random,page_touched,page_links_updated,page_latest,page_len,page_content_model,pr_page,pr_type,pr_level,pr_cascade,pr_user,pr_expiry,pr_id
0,25,0,Autism,NaN,558,0,0,0.626027,20160705210334,20160705210338,728503740,146470,wikitext,25,edit,autoconfirmed,0,NaN,infinity,373620
1,290,0,A,NaN,2377,0,0,0.854180,20160709170824,20160709170532,725198152,19399,wikitext,290,edit,autoconfirmed,0,NaN,infinity,394424
2,307,0,Abraham_Lincoln,NaN,8013,0,0,0.744777,20160709073542,20160709003852,728371848,171988,wikitext,307,edit,autoconfirmed,0,NaN,infinity,302458


In [50]:
# and convert them to STRINGS: this maps the values of the np.array into strings, and makes a list
ids2Qstr = list(map(str, ids2Q["page_id"].values)) 
print(len(ids2Qstr))

7526


In [51]:
",".join(ids2Qstr[(len(ids2Q)-5):])

'47779717,48421986,48750897,50109817,50730392'

In [52]:
# the dataframe obtained from the quarry function
# for PageLinks: "pl_from", "pagelinks", ["pl_from", "pl_title"]
# for categorylinks: "cl_from","categorylinks",["cl_from","cl_to"]

df_link = BatchQueryFromList(ids2Qstr,"cl_from","categorylinks",["cl_from","cl_to"])
len(df_link)

[09:03:18] fetching chunk 0
that was the first touch
[09:03:19] fetching chunk 1
here I'm appending        cl_from                                              cl_to
997        864                                     b'Gay_artists'
998        864                                     b'Gay_writers'
999        864                                  b'Hypochondriacs'
1000       864                            b'LGBT_Roman_Catholics'
1001       864             b'LGBT_artists_from_the_United_States'
1002       864                                  b'LGBT_directors'
1003       864                       b'LGBT_people_from_New_York'
1004       864                   b'LGBT_people_from_Pennsylvania'
1005       864                                  b'LGBT_producers'
1006       864             b'LGBT_writers_from_the_United_States'
1007       864                                    b'Male_artists'
1008       864                     b'Photographers_from_New_York'
1009       864                          b'

27897

In [53]:
df_link[(len(df_link)-5):]

,cl_from,cl_to
27892,39942,b'Wikipedia_articles_with_MusicBrainz_identifi...
27893,39942,b'Wikipedia_articles_with_SBN_identifiers'
27894,39942,b'Wikipedia_articles_with_VIAF_identifiers'
27895,39942,b'Wikipedia_indefinitely_move-protected_pages'
27896,39942,b'Wikipedia_indefinitely_semi-protected_biogra...


In [ ]:
# this decodes the links

link = df_link
link["pl_title"] = [x.decode("utf-8") for x in df_link["pl_title"].values]

In [ ]:
len(link)
link[:3]

In [ ]:
link.to_csv('pl_nepa_Batch1.csv', encoding="utf-8")

In [ ]:
saved_links = pd.read_csv("pl_nepa_Batch1.csv")

In [ ]:
del saved_links["Unnamed: 0"]
saved_links[:3]

In [ ]:
len(saved_links)
saved_links[(len(saved_links)-5):]

In [ ]:
art_num = set(saved_links["pl_from"])

In [ ]:
len(art_num)